## simple pdf to img

In [ ]:
%pip install chromadb pymupdf pillow gemma 

In [ ]:
%pip install ollama

In [ ]:
import fitz  # PyMuPDF for PDF -> images
import os
import base64
import json
import requests
import shutil


class PDFImageExtractor:
    """Convert PDF pages to images and manage temporary image folder."""

    def __init__(self, pdf_path, output_folder="pdf_images"):
        self.pdf_path = pdf_path
        self.output_folder = output_folder

    def pdf_to_images(self):
        """Convert each page of PDF to JPG image and return paths."""
        doc = fitz.open(self.pdf_path)
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

        image_paths = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=200)  
            image_path = os.path.join(self.output_folder, f"page_{page_num + 1}.jpg")
            pix.save(image_path)
            image_paths.append(image_path)
            print(f"✅ Page {page_num + 1} saved as {image_path}")
        doc.close()
        return image_paths

    def cleanup_images(self):
        """Delete all images from the output folder after extraction."""
        if os.path.exists(self.output_folder):
            shutil.rmtree(self.output_folder)
            print(f"🗑️ Deleted temporary folder: {self.output_folder}")


class GemmaOCR:
    """Send images to Gemma3 model via Ollama API to extract text."""

    def __init__(self, model="gemma3:12b", ollama_url="http://localhost:11434/api/generate"):
        self.model = model
        self.ollama_url = ollama_url

    def encode_image(self, image_path):
        """Convert image file to base64 string."""
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")

    def extract_text_from_image(self, image_path):
        """Send image to Gemma3 for OCR-like extraction."""
        image_b64 = self.encode_image(image_path)

        payload = {
            "model": self.model,
            "prompt": "Extract and return all readable text from this image.",
            "images": [image_b64]
        }

        try:
            
            response = requests.post(self.ollama_url, json=payload, stream=True)
            if response.status_code != 200:
                print(f"⚠️ Ollama API error {response.status_code}: {response.text}")
                return ""

            extracted_text = ""
            for line in response.iter_lines():
                if not line:
                    continue
                try:
                    data = json.loads(line.decode("utf-8"))
                    extracted_text += data.get("response") or data.get("content") or ""
                except json.JSONDecodeError:
                    continue  # skip malformed lines

            return extracted_text.strip()

        except Exception as e:
            print(f"⚠️ Error extracting text from image: {e}")
            return ""


def process_pdf_with_gemma(pdf_path):
    """Full pipeline: PDF -> images -> Gemma OCR -> combined text."""
    extractor = PDFImageExtractor(pdf_path)
    gemma_ocr = GemmaOCR()

    image_paths = extractor.pdf_to_images()
    all_text = ""

    for idx, image_path in enumerate(image_paths):
        print(f"\n🔍 Processing Page {idx + 1}...")
        page_text = gemma_ocr.extract_text_from_image(image_path)
        all_text += f"\n\n--- Page {idx + 1} ---\n{page_text}"

    
    extractor.cleanup_images()
    return all_text


if __name__ == "__main__":
    pdf_path = "./New folder/Return Policy.pdf"  # Update this to your PDF path
    output_text = process_pdf_with_gemma(pdf_path)

   
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_file = f"{base_name}.txt"
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(output_text)

    print(f"\n✅ Extraction Complete! Text saved to {output_file}")


## easy ocr

In [ ]:
#easy ocr from claude 1st attempt
import easyocr
import fitz  # PyMuPDF
from PIL import Image
import numpy as np
import os
from typing import List, Tuple
import cv2

class PDFOCRProcessor:
    def __init__(self, languages=['en'], gpu=False):
        """
        Initialize the PDF OCR processor
        
        Args:
            languages (list): List of languages for OCR (e.g., ['en', 'es'])
            gpu (bool): Whether to use GPU for OCR processing
        """
        print("Initializing EasyOCR...")
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("EasyOCR initialized successfully!")
    
    def pdf_to_images(self, pdf_path: str, output_dir: str = None, dpi: int = 200) -> List[str]:
        """
        Convert PDF pages to images
        
        Args:
            pdf_path (str): Path to the PDF file
            output_dir (str): Directory to save images (optional)
            dpi (int): DPI for image conversion (higher = better quality)
            
        Returns:
            List[str]: List of image file paths
        """
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
        
        # Create output directory if specified
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        
        # Open PDF
        doc = fitz.open(pdf_path)
        image_paths = []
        
        print(f"Converting {len(doc)} pages to images...")
        
        for page_num in range(len(doc)):
            
            page = doc[page_num]
            
            
            mat = fitz.Matrix(dpi/72, dpi/72)
            
            
            pix = page.get_pixmap(matrix=mat)
            
            
            img_data = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_data)) if 'io' in globals() else None
            
            
            if img is None:
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            
            if output_dir:
                image_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
                img.save(image_path, "PNG")
                image_paths.append(image_path)
            else:
                
                image_path = f"temp_page_{page_num + 1}.png"
                img.save(image_path, "PNG")
                image_paths.append(image_path)
            
            print(f"Page {page_num + 1} converted to image")
        
        doc.close()
        return image_paths
    
    def enhance_image_for_ocr(self, image_path: str) -> np.ndarray:
        """
        Enhance image quality for better OCR results
        
        Args:
            image_path (str): Path to the image file
            
        Returns:
            np.ndarray: Enhanced image array
        """
        
        img = cv2.imread(image_path)
        
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        
        denoised = cv2.fastNlMeansDenoising(gray)
        
        
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        
        return thresh
    
    def extract_text_from_image(self, image_path: str, enhance: bool = True) -> List[Tuple[str, float]]:
        """
        Extract text from image using EasyOCR
        
        Args:
            image_path (str): Path to the image file
            enhance (bool): Whether to enhance image before OCR
            
        Returns:
            List[Tuple[str, float]]: List of (text, confidence) tuples
        """
        if enhance:
            
            img = self.enhance_image_for_ocr(image_path)
            results = self.reader.readtext(img)
        else:
            
            results = self.reader.readtext(image_path)
        
        
        text_results = []
        for result in results:
            text = result[1]
            confidence = result[2]
            text_results.append((text, confidence))
        
        return text_results
    
    def process_pdf(self, pdf_path: str, output_dir: str = None, 
                   enhance_images: bool = True, confidence_threshold: float = 0.5,
                   cleanup_temp: bool = True) -> dict:
        """
        Complete pipeline: PDF -> Images -> OCR
        
        Args:
            pdf_path (str): Path to the PDF file
            output_dir (str): Directory to save images (optional)
            enhance_images (bool): Whether to enhance images for OCR
            confidence_threshold (float): Minimum confidence for text extraction
            cleanup_temp (bool): Whether to cleanup temporary image files
            
        Returns:
            dict: Results containing text from each page
        """
        results = {
            'pdf_path': pdf_path,
            'pages': [],
            'total_pages': 0,
            'extracted_text': ''
        }
        
        try:
            
            print("Step 1: Converting PDF to images...")
            image_paths = self.pdf_to_images(pdf_path, output_dir)
            results['total_pages'] = len(image_paths)
            
            
            print("Step 2: Extracting text from images...")
            all_text = []
            
            for i, image_path in enumerate(image_paths):
                print(f"Processing page {i + 1}/{len(image_paths)}...")
                
                
                text_results = self.extract_text_from_image(image_path, enhance_images)
                
                
                filtered_text = [text for text, conf in text_results if conf >= confidence_threshold]
                
                page_text = ' '.join(filtered_text)
                all_text.append(page_text)
                
                page_info = {
                    'page_number': i + 1,
                    'image_path': image_path,
                    'text': page_text,
                    'raw_results': text_results
                }
                results['pages'].append(page_info)
            
            
            results['extracted_text'] = '\n\n--- Page Break ---\n\n'.join(all_text)
            
            
            if cleanup_temp and not output_dir:
                for image_path in image_paths:
                    if os.path.exists(image_path) and image_path.startswith('temp_'):
                        os.remove(image_path)
                        print(f"Cleaned up temporary file: {image_path}")
            
            print("Processing completed successfully!")
            
        except Exception as e:
            print(f"Error during processing: {str(e)}")
            results['error'] = str(e)
        
        return results

# Example usage
def main():
    """
    Example usage of the PDFOCRProcessor
    """
    
    processor = PDFOCRProcessor(languages=['en'], gpu=False)
    
    
    pdf_path = "./New folder/privacy_policy.pdf"  
    
    # Optional: specify output directory for images
    output_dir = "extracted_images"  # Set to None to use temporary files
    
    # Process the PDF
    results = processor.process_pdf(
        pdf_path=pdf_path,
        output_dir=output_dir,
        enhance_images=True,
        confidence_threshold=0.5,
        cleanup_temp=True
    )
    
    # Print results
    if 'error' not in results:
        print(f"\n{'='*50}")
        print(f"EXTRACTION RESULTS")
        print(f"{'='*50}")
        print(f"Total pages processed: {results['total_pages']}")
        print(f"\nExtracted Text:\n{'-'*30}")
        print(results['extracted_text'])
        
        # Save extracted text to file
        with open('extracted_text.txt', 'w', encoding='utf-8') as f:
            f.write(results['extracted_text'])
        print(f"\nExtracted text saved to: extracted_text.txt")
        
        # Print page-wise details
        print(f"\n{'='*50}")
        print("PAGE-WISE DETAILS")
        print(f"{'='*50}")
        for page in results['pages']:
            print(f"\nPage {page['page_number']}:")
            print(f"Text: {page['text'][:200]}..." if len(page['text']) > 200 else f"Text: {page['text']}")
    else:
        print(f"Error occurred: {results['error']}")

if __name__ == "__main__":
    # Import required libraries check
    try:
        import io
    except ImportError:
        import io
    
    main()

"""
Installation requirements:
pip install easyocr PyMuPDF pillow opencv-python numpy

Usage:
1. Install the required packages
2. Replace 'sample_document.pdf' with your PDF path
3. Run the script
4. The extracted text will be saved to 'extracted_text.txt'

Features:
- Converts PDF pages to high-resolution images
- Enhances images for better OCR accuracy
- Extracts text with confidence scores
- Filters results by confidence threshold
- Supports multiple languages
- Saves extracted text to file
- Option to keep or cleanup temporary image files
"""

#it worked

Using CPU. Note: This module is much faster with a GPU.


Initializing EasyOCR...
EasyOCR initialized successfully!
Step 1: Converting PDF to images...
Converting 7 pages to images...
Page 1 converted to image
Page 2 converted to image
Page 3 converted to image
Page 4 converted to image
Page 5 converted to image
Page 6 converted to image
Page 7 converted to image
Step 2: Extracting text from images...
Processing page 1/7...


c:\Users\Busra\Melikoz\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Processing page 2/7...
Processing page 3/7...
Processing page 4/7...
Processing page 5/7...
Processing page 6/7...
Processing page 7/7...
Processing completed successfully!

EXTRACTION RESULTS
Total pages processed: 7

Extracted Text:
------------------------------
8/25/25,5.19 PM Calmly Writer Online 1 Introduction This privacy policy describes the personal data that Shopify processes (which may include collecting, organizing, structuring, storing, using, or disclosing) when you use or access Shopify Consumer Services, or when you interact with a merchant that is powered by Shopify's platform ("Shopify Merchants" or "Merchants") . If any provisions in this policy conflict with provisions of any other Shopify privacy policy, the provisions in this policy will apply: 2. What Personal Data We Receive Personal Data is any information from or about an identified or identifiable person; but does not include aggregated or anonymized information. We receive the following types of Personal Dat

"\nInstallation requirements:\npip install easyocr PyMuPDF pillow opencv-python numpy\n\nUsage:\n1. Install the required packages\n2. Replace 'sample_document.pdf' with your PDF path\n3. Run the script\n4. The extracted text will be saved to 'extracted_text.txt'\n\nFeatures:\n- Converts PDF pages to high-resolution images\n- Enhances images for better OCR accuracy\n- Extracts text with confidence scores\n- Filters results by confidence threshold\n- Supports multiple languages\n- Saves extracted text to file\n- Option to keep or cleanup temporary image files\n"

## chromaDB embedding

In [ ]:
import chromadb
import os
from chromadb.config import Settings

class TextEmbedder:
    def __init__(self, db_path="./chroma_db"):
        # Initialize ChromaDB client
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        
        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )
    
    def chunk_text(self, text, chunk_size=500, overlap=50):
        """Split text into overlapping chunks."""
        chunks = []
        words = text.split()
        
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk.strip())
        
        return chunks
    
    def embed_text_file(self, txt_file_path):
        """Read text file and embed it chunk by chunk."""
        if not os.path.exists(txt_file_path):
            print(f"❌ File not found: {txt_file_path}")
            return
        
        
        with open(txt_file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        
        chunks = self.chunk_text(text)
        
        
        documents = chunks
        ids = [f"chunk_{i+1}" for i in range(len(chunks))]
        metadatas = [{"source": txt_file_path, "chunk_index": i+1} for i in range(len(chunks))]
        
        
        self.collection.add(
            documents=documents,
            ids=ids,
            metadatas=metadatas
        )
        
        print(f"✅ Successfully embedded {len(chunks)} chunks from {txt_file_path}")
        print(f"💾 Stored in collection: {self.collection_name}")
        return len(chunks)
    
    def view_embeddings(self):
        """Display all embedded documents."""
        results = self.collection.get()
        
        print(f"\n📊 Collection: {self.collection_name}")
        print(f"📈 Total embeddings: {len(results['ids'])}")
        print("\n" + "="*80)
        
        for i, (doc_id, document, metadata) in enumerate(zip(
            results['ids'], 
            results['documents'], 
            results['metadatas']
        )):
            print(f"\n🔹 ID: {doc_id}")
            print(f"📄 Source: {metadata['source']}")
            print(f"📝 Content Preview: {document[:200]}...")
            print("-" * 80)
    
    def search_similar(self, query, n_results=3):
        """Search for similar chunks."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        
        print(f"\n🔍 Search Results for: '{query}'")
        print("="*80)
        
        for i, (doc_id, document, metadata, distance) in enumerate(zip(
            results['ids'][0], 
            results['documents'][0], 
            results['metadatas'][0],
            results['distances'][0]
        )):
            print(f"\n{i+1}. ID: {doc_id}")
            print(f"   Distance: {distance:.4f}")
            print(f"   Content: {document[:300]}...")
            print("-" * 80)


def main():
    # Initialize embedder
    embedder = TextEmbedder()
    
    # Embed your text file (change this to your actual file)
    txt_file = "extracted_text.txt"  # Change to your actual txt file name
    
    if os.path.exists(txt_file):
        embedder.embed_text_file(txt_file)
        
        # View all embeddings
        embedder.view_embeddings()
        
        # Example search (optional)
        print("\n" + "="*80)
        search_query = input("Enter search query (or press Enter to skip): ").strip()
        if search_query:
            embedder.search_similar(search_query)
    else:
        print(f"❌ Text file '{txt_file}' not found!")
        print("Available txt files:")
        for file in os.listdir("."):
            if file.endswith(".txt"):
                print(f"  - {file}")


if __name__ == "__main__":
    main()

#it worked embedding done

✅ Successfully embedded 5 chunks from extracted_text.txt
💾 Stored in collection: pdf_embeddings

📊 Collection: pdf_embeddings
📈 Total embeddings: 5


🔹 ID: chunk_1
📄 Source: extracted_text.txt
📝 Content Preview: 8/25/25,5.19 PM Calmly Writer Online 1 Introduction This privacy policy describes the personal data that Shopify processes (which may include collecting, organizing, structuring, storing, using, or di...
--------------------------------------------------------------------------------

🔹 ID: chunk_2
📄 Source: extracted_text.txt
📝 Content Preview: as information about your browser and device privacy settings: 7 . Communications with Shopify: We receive information, including message and email content; when you communicate with Shopify (such as ...
--------------------------------------------------------------------------------

🔹 ID: chunk_3
📄 Source: extracted_text.txt
📝 Content Preview: to detect; investigate, and prevent malicious conduct; fraudulent activity or unsafe experie

## Chroma with structued.md

In [ ]:
import chromadb
import os
from chromadb.config import Settings

class TextEmbedder:
    def __init__(self, db_path="./chroma_db_structured"):
        # Initialize ChromaDB client
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        
        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )
    
    def chunk_text(self, text, chunk_size=500, overlap=50):
        """Split text into overlapping chunks."""
        chunks = []
        words = text.split()
        
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk.strip())
        
        return chunks
    
    def embed_text(self, text: str, source: str = "structured_output.md"):
        """Embed the text provided directly (from structured_output.md)."""
        # Split into chunks
        chunks = self.chunk_text(text)
        
        # Prepare data for ChromaDB
        documents = chunks
        ids = [f"chunk_{i+1}" for i in range(len(chunks))]
        metadatas = [{"source": source, "chunk_index": i+1} for i in range(len(chunks))]
        
        # Add to ChromaDB
        self.collection.add(
            documents=documents,
            ids=ids,
            metadatas=metadatas
        )
        
        print(f"✅ Successfully embedded {len(chunks)} chunks from {source}")
        print(f"💾 Stored in collection: {self.collection_name}")
        return len(chunks)
    
    def view_embeddings(self):
        """Display all embedded documents."""
        results = self.collection.get()
        
        print(f"\n📊 Collection: {self.collection_name}")
        print(f"📈 Total embeddings: {len(results['ids'])}")
        print("\n" + "="*80)
        
        for i, (doc_id, document, metadata) in enumerate(zip(
            results['ids'], 
            results['documents'], 
            results['metadatas']
        )):
            print(f"\n🔹 ID: {doc_id}")
            print(f"📄 Source: {metadata['source']}")
            print(f"📝 Content Preview: {document[:200]}...")
            print("-" * 80)
    
    def search_similar(self, query, n_results=3):
        """Search for similar chunks."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        
        print(f"\n🔍 Search Results for: '{query}'")
        print("="*80)
        
        for i, (doc_id, document, metadata, distance) in enumerate(zip(
            results['ids'][0], 
            results['documents'][0], 
            results['metadatas'][0],
            results['distances'][0]
        )):
            print(f"\n{i+1}. ID: {doc_id}")
            print(f"   Distance: {distance:.4f}")
            print(f"   Content: {document[:300]}...")
            print("-" * 80)


def main():
    # Initialize embedder
    embedder = TextEmbedder()
    
    # Read structured output from the markdown file
    txt_file_path = "structured_output1.md"  # The OCR output file
    
    if os.path.exists(txt_file_path):
        with open(txt_file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        # Embed the structured content from the file
        embedder.embed_text(text, source=txt_file_path)
        
        # View all embeddings
        embedder.view_embeddings()
        
        # Example search (optional)
        print("\n" + "="*80)
        search_query = input("Enter search query (or press Enter to skip): ").strip()
        if search_query:
            embedder.search_similar(search_query)
    else:
        print(f"❌ Text file '{txt_file_path}' not found!")
        print("Available txt files:")
        for file in os.listdir("."):
            if file.endswith(".md"):  # Looking for markdown files
                print(f"  - {file}")


if __name__ == "__main__":
    main()


#it worked embedding done structured

✅ Successfully embedded 5 chunks from structured_output1.md
💾 Stored in collection: pdf_embeddings

📊 Collection: pdf_embeddings
📈 Total embeddings: 5


🔹 ID: chunk_1
📄 Source: structured_output1.md
📝 Content Preview: --- Page 1 --- # Calmly Writer Online # 8/25/25, 5.19 PM # Introduction # 1 # This privacy policy describes the personal data that Shopify processes (which may include # collecting, organizing, struct...
--------------------------------------------------------------------------------

🔹 ID: chunk_2
📄 Source: structured_output1.md
📝 Content Preview: transaction # details and your payment information. Settings and Privacy Preferences: We receive information about your settings and 6. preferences on Shopify and our Merchants, as well as information...
--------------------------------------------------------------------------------

🔹 ID: chunk_3
📄 Source: structured_output1.md
📝 Content Preview: information to respond to you when you contact uS. 6. Authentication, Integrity,

## easyOCR versions

In [ ]:
#1
import easyocr
import fitz  # PyMuPDF
from PIL import Image
import numpy as np
import os
from typing import List, Tuple
import cv2

class PDFOCRProcessor:
    def __init__(self, languages=['en'], gpu=False):
        """
        Initialize the PDF OCR processor
        
        Args:
            languages (list): List of languages for OCR (e.g., ['en', 'es'])
            gpu (bool): Whether to use GPU for OCR processing
        """
        print("Initializing EasyOCR...")
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("EasyOCR initialized successfully!")
    
    def pdf_to_images(self, pdf_path: str, output_dir: str = None, dpi: int = 200) -> List[str]:
        """
        Convert PDF pages to images
        
        Args:
            pdf_path (str): Path to the PDF file
            output_dir (str): Directory to save images (optional)
            dpi (int): DPI for image conversion (higher = better quality)
            
        Returns:
            List[str]: List of image file paths
        """
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
        
        # Create output directory if specified
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        
        # Open PDF
        doc = fitz.open(pdf_path)
        image_paths = []
        
        print(f"Converting {len(doc)} pages to images...")
        
        for page_num in range(len(doc)):
            # Get page
            page = doc[page_num]
            
            # Create transformation matrix for higher resolution
            mat = fitz.Matrix(dpi/72, dpi/72)
            
            # Render page as image
            pix = page.get_pixmap(matrix=mat)
            
            # Convert to PIL Image
            img_data = pix.tobytes("png")
            img = Image.open(io.BytesIO(img_data)) if 'io' in globals() else None
            
            # Alternative method using pix.pil_tobytes()
            if img is None:
                img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            # Save image
            if output_dir:
                image_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
                img.save(image_path, "PNG")
                image_paths.append(image_path)
            else:
                # Save to temporary location
                image_path = f"temp_page_{page_num + 1}.png"
                img.save(image_path, "PNG")
                image_paths.append(image_path)
            
            print(f"Page {page_num + 1} converted to image")
        
        doc.close()
        return image_paths
    
    def enhance_image_for_ocr(self, image_path: str) -> np.ndarray:
        """
        Enhance image quality for better OCR results
        
        Args:
            image_path (str): Path to the image file
            
        Returns:
            np.ndarray: Enhanced image array
        """
        # Read image
        img = cv2.imread(image_path)
        
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply denoising
        denoised = cv2.fastNlMeansDenoising(gray)
        
        # Apply adaptive thresholding for better contrast
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                       cv2.THRESH_BINARY, 11, 2)
        
        return thresh
    
    def extract_text_from_image(self, image_path: str, enhance: bool = True) -> List[Tuple[str, float]]:
        """
        Extract text from image using EasyOCR
        
        Args:
            image_path (str): Path to the image file
            enhance (bool): Whether to enhance image before OCR
            
        Returns:
            List[Tuple[str, float]]: List of (text, confidence) tuples
        """
        if enhance:
            # Use enhanced image
            img = self.enhance_image_for_ocr(image_path)
            results = self.reader.readtext(img)
        else:
            # Use original image
            results = self.reader.readtext(image_path)
        
        # Extract text and confidence scores
        text_results = []
        for result in results:
            text = result[1]
            confidence = result[2]
            text_results.append((text, confidence))
        
        return text_results
    
    def process_pdf(self, pdf_path: str, output_dir: str = None, 
                   enhance_images: bool = True, confidence_threshold: float = 0.5,
                   cleanup_temp: bool = True) -> dict:
        """
        Complete pipeline: PDF -> Images -> OCR
        
        Args:
            pdf_path (str): Path to the PDF file
            output_dir (str): Directory to save images (optional)
            enhance_images (bool): Whether to enhance images for OCR
            confidence_threshold (float): Minimum confidence for text extraction
            cleanup_temp (bool): Whether to cleanup temporary image files
            
        Returns:
            dict: Results containing text from each page
        """
        results = {
            'pdf_path': pdf_path,
            'pages': [],
            'total_pages': 0,
            'extracted_text': ''
        }
        
        try:
            # Step 1: Convert PDF to images
            print("Step 1: Converting PDF to images...")
            image_paths = self.pdf_to_images(pdf_path, output_dir)
            results['total_pages'] = len(image_paths)
            
            # Step 2: Extract text from each image
            print("Step 2: Extracting text from images...")
            all_text = []
            
            for i, image_path in enumerate(image_paths):
                print(f"Processing page {i + 1}/{len(image_paths)}...")
                
                # Extract text
                text_results = self.extract_text_from_image(image_path, enhance_images)
                
                # Filter by confidence threshold
                filtered_text = [text for text, conf in text_results if conf >= confidence_threshold]
                
                page_text = ' '.join(filtered_text)
                all_text.append(page_text)
                
                page_info = {
                    'page_number': i + 1,
                    'image_path': image_path,
                    'text': page_text,
                    'raw_results': text_results
                }
                results['pages'].append(page_info)
            
            # Combine all text
            results['extracted_text'] = '\n\n--- Page Break ---\n\n'.join(all_text)
            
            # Cleanup temporary files if requested
            if cleanup_temp and not output_dir:
                for image_path in image_paths:
                    if os.path.exists(image_path) and image_path.startswith('temp_'):
                        os.remove(image_path)
                        print(f"Cleaned up temporary file: {image_path}")
            
            print("Processing completed successfully!")
            
        except Exception as e:
            print(f"Error during processing: {str(e)}")
            results['error'] = str(e)
        
        return results

# Example usage
def main():
    """
    Example usage of the PDFOCRProcessor
    """
    # Initialize processor
    # For multiple languages, use: languages=['en', 'es', 'fr']
    processor = PDFOCRProcessor(languages=['en'], gpu=False)
    
    # Path to your PDF file
    pdf_path = "sample_document.pdf"  # Change this to your PDF path
    
    # Optional: specify output directory for images
    output_dir = "extracted_images"  # Set to None to use temporary files
    
    # Process the PDF
    results = processor.process_pdf(
        pdf_path=pdf_path,
        output_dir=output_dir,
        enhance_images=True,
        confidence_threshold=0.5,
        cleanup_temp=True
    )
    
    # Print results
    if 'error' not in results:
        print(f"\n{'='*50}")
        print(f"EXTRACTION RESULTS")
        print(f"{'='*50}")
        print(f"Total pages processed: {results['total_pages']}")
        print(f"\nExtracted Text:\n{'-'*30}")
        print(results['extracted_text'])
        
        # Save extracted text to file
        with open('extracted_text.txt', 'w', encoding='utf-8') as f:
            f.write(results['extracted_text'])
        print(f"\nExtracted text saved to: extracted_text.txt")
        
        # Print page-wise details
        print(f"\n{'='*50}")
        print("PAGE-WISE DETAILS")
        print(f"{'='*50}")
        for page in results['pages']:
            print(f"\nPage {page['page_number']}:")
            print(f"Text: {page['text'][:200]}..." if len(page['text']) > 200 else f"Text: {page['text']}")
    else:
        print(f"Error occurred: {results['error']}")

if __name__ == "__main__":
    # Import required libraries check
    try:
        import io
    except ImportError:
        import io
    
    main()

"""
Installation requirements:
pip install easyocr PyMuPDF pillow opencv-python numpy

Usage:
1. Install the required packages
2. Replace 'sample_document.pdf' with your PDF path
3. Run the script
4. The extracted text will be saved to 'extracted_text.txt'

Features:
- Converts PDF pages to high-resolution images
- Enhances images for better OCR accuracy
- Extracts text with confidence scores
- Filters results by confidence threshold
- Supports multiple languages
- Saves extracted text to file
- Option to keep or cleanup temporary image files
"""

In [2]:
#2
import easyocr
import fitz  # PyMuPDF
from PIL import Image
import numpy as np
import cv2
import pandas as pd
import json
from typing import List, Dict, Tuple, Any
import os
import io
from dataclasses import dataclass

@dataclass
class TextBlock:
    text: str
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    confidence: float
    block_type: str = "text"  # text, table, header, footer

class AdvancedPDFOCR:
    def __init__(self, languages=['en'], gpu=False):
        """
        Advanced PDF OCR processor with structure preservation
        """
        print("Initializing EasyOCR...")
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("EasyOCR initialized successfully!")
    
    def pdf_to_images(self, pdf_path: str, output_dir: str = None, dpi: int = 300) -> List[str]:
        """Convert PDF to high-quality images"""
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"PDF file not found: {pdf_path}")
        
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
        
        doc = fitz.open(pdf_path)
        image_paths = []
        
        print(f"Converting {len(doc)} pages to images...")
        
        for page_num in range(len(doc)):
            page = doc[page_num]
            mat = fitz.Matrix(dpi/72, dpi/72)
            pix = page.get_pixmap(matrix=mat)
            
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            if output_dir:
                image_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
            else:
                image_path = f"temp_page_{page_num + 1}.png"
            
            img.save(image_path, "PNG", dpi=(dpi, dpi))
            image_paths.append(image_path)
            print(f"Page {page_num + 1} converted (DPI: {dpi})")
        
        doc.close()
        return image_paths
    
    def detect_text_blocks(self, image_path: str) -> List[TextBlock]:
        """
        Detect text blocks with bounding boxes for structure analysis
        """
        img = cv2.imread(image_path)
        results = self.reader.readtext(img)
        
        text_blocks = []
        for result in results:
            bbox_points = result[0]  # 4 corner points
            text = result[1]
            confidence = result[2]
            
            # Convert bbox points to rectangle coordinates
            x_coords = [point[0] for point in bbox_points]
            y_coords = [point[1] for point in bbox_points]
            x1, y1, x2, y2 = min(x_coords), min(y_coords), max(x_coords), max(y_coords)
            
            block = TextBlock(text, (x1, y1, x2, y2), confidence)
            text_blocks.append(block)
        
        return text_blocks
    
    def analyze_layout(self, text_blocks: List[TextBlock], img_shape: Tuple[int, int]) -> List[TextBlock]:
        """
        Analyze layout to identify headers, tables, and text sections
        """
        height, width = img_shape[:2]
        
        # Sort blocks by vertical position
        sorted_blocks = sorted(text_blocks, key=lambda b: b.bbox[1])
        
        for i, block in enumerate(sorted_blocks):
            x1, y1, x2, y2 = block.bbox
            
            # Identify potential headers (top 20% of page, larger text)
            if y1 < height * 0.2:
                block.block_type = "header"
            
            # Identify potential footers (bottom 10% of page)
            elif y1 > height * 0.9:
                block.block_type = "footer"
            
            # Detect potential table content (aligned text in columns)
            elif self._is_table_like(block, sorted_blocks, i):
                block.block_type = "table"
            
            else:
                block.block_type = "text"
        
        return sorted_blocks
    
    def _is_table_like(self, block: TextBlock, all_blocks: List[TextBlock], index: int) -> bool:
        """
        Simple heuristic to detect table-like content
        """
        x1, y1, x2, y2 = block.bbox
        
        # Look for blocks with similar y-coordinates (same row)
        same_row_blocks = []
        for other_block in all_blocks:
            ox1, oy1, ox2, oy2 = other_block.bbox
            # If vertically aligned (similar y-coordinates)
            if abs(y1 - oy1) < 10 and abs(y2 - oy2) < 10 and other_block != block:
                same_row_blocks.append(other_block)
        
        # If we have multiple blocks in the same row, it might be a table
        return len(same_row_blocks) >= 2
    
    def reconstruct_structure(self, text_blocks: List[TextBlock], preserve_format: str = "markdown") -> str:
        """
        Reconstruct text with preserved structure
        """
        if preserve_format == "markdown":
            return self._to_markdown(text_blocks)
        elif preserve_format == "json":
            return self._to_json(text_blocks)
        elif preserve_format == "xml":
            return self._to_xml(text_blocks)
        else:
            return self._to_structured_text(text_blocks)
    
    def _to_markdown(self, text_blocks: List[TextBlock]) -> str:
        """Convert to Markdown with preserved structure"""
        markdown_content = []
        current_table_rows = []
        in_table = False
        
        for block in text_blocks:
            if block.block_type == "header":
                if in_table and current_table_rows:
                    markdown_content.extend(self._format_table_markdown(current_table_rows))
                    current_table_rows = []
                    in_table = False
                markdown_content.append(f"# {block.text}")
            
            elif block.block_type == "table":
                if not in_table:
                    in_table = True
                current_table_rows.append(block)
            
            elif block.block_type == "footer":
                if in_table and current_table_rows:
                    markdown_content.extend(self._format_table_markdown(current_table_rows))
                    current_table_rows = []
                    in_table = False
                markdown_content.append(f"*{block.text}*")
            
            else:  # regular text
                if in_table and current_table_rows:
                    markdown_content.extend(self._format_table_markdown(current_table_rows))
                    current_table_rows = []
                    in_table = False
                markdown_content.append(block.text)
        
        # Handle remaining table rows
        if in_table and current_table_rows:
            markdown_content.extend(self._format_table_markdown(current_table_rows))
        
        return "\n\n".join(markdown_content)
    
    def _format_table_markdown(self, table_blocks: List[TextBlock]) -> List[str]:
        """Format table blocks as Markdown table"""
        if not table_blocks:
            return []
        
        # Group blocks by rows based on y-coordinates
        rows = []
        current_row = []
        current_y = table_blocks[0].bbox[1]
        
        for block in sorted(table_blocks, key=lambda b: (b.bbox[1], b.bbox[0])):
            if abs(block.bbox[1] - current_y) > 20:  # New row
                if current_row:
                    rows.append(current_row)
                current_row = [block]
                current_y = block.bbox[1]
            else:
                current_row.append(block)
        
        if current_row:
            rows.append(current_row)
        
        if not rows:
            return []
        
        # Convert to markdown table
        markdown_rows = []
        for i, row in enumerate(rows):
            row_text = " | ".join([block.text for block in row])
            markdown_rows.append(f"| {row_text} |")
            
            # Add header separator after first row
            if i == 0:
                separator = " | ".join(["---"] * len(row))
                markdown_rows.append(f"| {separator} |")
        
        return markdown_rows
    
    def _to_json(self, text_blocks: List[TextBlock]) -> str:
        """Convert to JSON structure"""
        json_data = {
            "document_structure": [],
            "metadata": {
                "total_blocks": len(text_blocks),
                "block_types": {}
            }
        }
        
        for block in text_blocks:
            block_data = {
                "text": block.text,
                "type": block.block_type,
                "bbox": block.bbox,
                "confidence": block.confidence
            }
            json_data["document_structure"].append(block_data)
            
            # Update metadata
            if block.block_type in json_data["metadata"]["block_types"]:
                json_data["metadata"]["block_types"][block.block_type] += 1
            else:
                json_data["metadata"]["block_types"][block.block_type] = 1
        
        return json.dumps(json_data, indent=2, ensure_ascii=False)
    
    def _to_xml(self, text_blocks: List[TextBlock]) -> str:
        """Convert to XML structure"""
        xml_content = ['<?xml version="1.0" encoding="UTF-8"?>', '<document>']
        
        for block in text_blocks:
            x1, y1, x2, y2 = block.bbox
            xml_content.append(
                f'  <{block.block_type} '
                f'bbox="{x1},{y1},{x2},{y2}" '
                f'confidence="{block.confidence:.2f}">'
            )
            xml_content.append(f'    {block.text}')
            xml_content.append(f'  </{block.block_type}>')
        
        xml_content.append('</document>')
        return '\n'.join(xml_content)
    
    def _to_structured_text(self, text_blocks: List[TextBlock]) -> str:
        """Convert to structured plain text"""
        structured_content = []
        
        for block in text_blocks:
            if block.block_type == "header":
                structured_content.append(f"\n[HEADER] {block.text}")
            elif block.block_type == "table":
                structured_content.append(f"[TABLE_CELL] {block.text}")
            elif block.block_type == "footer":
                structured_content.append(f"[FOOTER] {block.text}")
            else:
                structured_content.append(block.text)
        
        return "\n".join(structured_content)
    
    def create_embeddings_ready_chunks(self, structured_text: str, chunk_size: int = 512) -> List[Dict[str, Any]]:
        """
        Create embedding-ready chunks with context preservation
        """
        chunks = []
        
        # Split by major sections while preserving structure
        sections = structured_text.split('\n\n')
        current_chunk = ""
        chunk_metadata = {"type": "mixed", "contains_table": False, "contains_header": False}
        
        for section in sections:
            # Check if adding this section would exceed chunk size
            if len(current_chunk + section) > chunk_size and current_chunk:
                # Save current chunk
                chunks.append({
                    "text": current_chunk.strip(),
                    "metadata": chunk_metadata.copy(),
                    "length": len(current_chunk)
                })
                current_chunk = section
                chunk_metadata = {"type": "mixed", "contains_table": False, "contains_header": False}
            else:
                current_chunk += "\n\n" + section if current_chunk else section
            
            # Update metadata based on content
            if section.startswith("#") or "[HEADER]" in section:
                chunk_metadata["contains_header"] = True
            if "|" in section or "[TABLE_CELL]" in section:
                chunk_metadata["contains_table"] = True
        
        # Add final chunk
        if current_chunk:
            chunks.append({
                "text": current_chunk.strip(),
                "metadata": chunk_metadata,
                "length": len(current_chunk)
            })
        
        return chunks
    
    def process_pdf_advanced(self, pdf_path: str, output_format: str = "markdown", 
                           chunk_for_embeddings: bool = True, confidence_threshold: float = 0.5) -> Dict[str, Any]:
        """
        Advanced PDF processing with structure preservation
        """
        results = {
            "pdf_path": pdf_path,
            "pages": [],
            "structured_content": "",
            "embedding_chunks": [],
            "metadata": {}
        }
        
        try:
            # Convert PDF to images
            print("Step 1: Converting PDF to images...")
            image_paths = self.pdf_to_images(pdf_path)
            
            all_structured_content = []
            
            # Process each page
            for i, image_path in enumerate(image_paths):
                print(f"Step 2: Processing page {i + 1}/{len(image_paths)}...")
                
                # Detect text blocks with positions
                text_blocks = self.detect_text_blocks(image_path)
                
                # Filter by confidence
                filtered_blocks = [b for b in text_blocks if b.confidence >= confidence_threshold]
                
                # Analyze layout
                img = cv2.imread(image_path)
                structured_blocks = self.analyze_layout(filtered_blocks, img.shape)
                
                # Reconstruct with structure
                page_content = self.reconstruct_structure(structured_blocks, output_format)
                all_structured_content.append(f"--- Page {i + 1} ---\n{page_content}")
                
                # Store page info
                results["pages"].append({
                    "page_number": i + 1,
                    "blocks": len(structured_blocks),
                    "content": page_content,
                    "block_types": {block.block_type: 1 for block in structured_blocks}
                })
            
            # Combine all content
            results["structured_content"] = "\n\n".join(all_structured_content)
            
            # Create embedding chunks if requested
            if chunk_for_embeddings:
                print("Step 3: Creating embedding-ready chunks...")
                results["embedding_chunks"] = self.create_embeddings_ready_chunks(
                    results["structured_content"]
                )
            
            # Add metadata
            results["metadata"] = {
                "total_pages": len(image_paths),
                "output_format": output_format,
                "total_chunks": len(results["embedding_chunks"]) if chunk_for_embeddings else 0,
                "processing_complete": True
            }
            
            # Cleanup temp files
            for image_path in image_paths:
                if image_path.startswith('temp_'):
                    os.remove(image_path)
            print("Processing completed successfully!")
            
        except Exception as e:
            print(f"Error: {str(e)}")
            results["error"] = str(e)
        
        return results

# Usage examples
def demo_usage():
    """
    Demonstration of different output formats and embedding preparation
    """
    processor = AdvancedPDFOCR(languages=['en'], gpu=False)
    pdf_path = "./New folder/privacy_policy.pdf"  # Your PDF path
    
    print("=" * 60)
    print("PROCESSING WITH DIFFERENT FORMATS")
    print("=" * 60)
    
    # Process with Markdown format (best for tables)
    print("\n1. MARKDOWN FORMAT (Best for tables and structure)")
    results_md = processor.process_pdf_advanced(
        pdf_path, 
        output_format="markdown", 
        chunk_for_embeddings=True
    )
    
    if "error" not in results_md:
        print("Structured Content Preview:")
        print(results_md["structured_content"][:1000] + "...")
        
        # Save outputs
        with open("structured_output1.md", "w", encoding="utf-8") as f:
            f.write(results_md["structured_content"])
        
        # Save embedding chunks
        with open("embedding_chunks.json", "w", encoding="utf-8") as f:
            json.dump(results_md["embedding_chunks"], f, indent=2, ensure_ascii=False)
        
        print(f"\nFiles saved:")
        print(f"- Structured content: structured_output1.md")
        print(f"- Embedding chunks: embedding_chunks.json")
        print(f"- Total chunks for embeddings: {len(results_md['embedding_chunks'])}")
    
    # Process with JSON format (best for programmatic access)
    print("\n2. JSON FORMAT (Best for programmatic processing)")
    results_json = processor.process_pdf_advanced(
        pdf_path, 
        output_format="json", 
        chunk_for_embeddings=False
    )
    
    if "error" not in results_json:
        with open("structured_output1.json", "w", encoding="utf-8") as f:
            f.write(results_json["structured_content"])
        print("JSON structure saved to: structured_output1.json")

if __name__ == "__main__":
    demo_usage()

"""
RECOMMENDED APPROACHES FOR EMBEDDINGS:

1. **For Financial Documents (like credit reports):**
   - Use Markdown format to preserve table structure
   - Create chunks that respect logical boundaries (complete sections)
   - Include metadata about content type (header, table, text)

2. **Best Practices:**
   - DPI 300+ for better text detection
   - Structure-aware chunking (don't split tables)
   - Preserve context with section headers
   - Include bounding box information for spatial understanding

3. **For Better Embeddings:**
   - Keep related information together (account details + payment history)
   - Use semantic boundaries rather than fixed character limits
   - Include document structure markers
   - Preserve table relationships

Installation:
pip install easyocr PyMuPDF pillow opencv-python pandas numpy
"""


#structure_output.md

Using CPU. Note: This module is much faster with a GPU.


Initializing EasyOCR...
EasyOCR initialized successfully!
PROCESSING WITH DIFFERENT FORMATS

1. MARKDOWN FORMAT (Best for tables and structure)
Step 1: Converting PDF to images...
Converting 7 pages to images...
Page 1 converted (DPI: 300)
Page 2 converted (DPI: 300)
Page 3 converted (DPI: 300)
Page 4 converted (DPI: 300)
Page 5 converted (DPI: 300)
Page 6 converted (DPI: 300)
Page 7 converted (DPI: 300)
Step 2: Processing page 1/7...


c:\Users\Busra\Melikoz\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step 2: Processing page 2/7...
Step 2: Processing page 3/7...
Step 2: Processing page 4/7...
Step 2: Processing page 5/7...
Step 2: Processing page 6/7...
Step 2: Processing page 7/7...
Step 3: Creating embedding-ready chunks...
Processing completed successfully!
Structured Content Preview:
--- Page 1 ---
# Calmly Writer Online

# 8/25/25, 5.19 PM

# Introduction

# 1

# This privacy policy describes the personal data that Shopify processes (which may include

# collecting, organizing, structuring, storing, using, or disclosing) when you use or access

# Shopify Consumer Services, or when you interact with a merchant that is powered by

# Shopify's platform ("Shopify Merchants" or "Merchants") . If any provisions in this policy

conflict with provisions of any other Shopify privacy policy, the provisions in this policy will

apply:

2. What Personal Data We Receive

Personal Data is any information from or about an identified or identifiable person, but does

not include aggregated or 

"\nRECOMMENDED APPROACHES FOR EMBEDDINGS:\n\n1. **For Financial Documents (like credit reports):**\n   - Use Markdown format to preserve table structure\n   - Create chunks that respect logical boundaries (complete sections)\n   - Include metadata about content type (header, table, text)\n\n2. **Best Practices:**\n   - DPI 300+ for better text detection\n   - Structure-aware chunking (don't split tables)\n   - Preserve context with section headers\n   - Include bounding box information for spatial understanding\n\n3. **For Better Embeddings:**\n   - Keep related information together (account details + payment history)\n   - Use semantic boundaries rather than fixed character limits\n   - Include document structure markers\n   - Preserve table relationships\n\nInstallation:\npip install easyocr PyMuPDF pillow opencv-python pandas numpy\n"

In [ ]:
#HAVE TO GPT THIS FOR ACHEIVE GOOD OCR RESULT


# Using CPU. Note: This module is much faster with a GPU.
# Initializing EasyOCR...
# EasyOCR initialized successfully!
# ============================================================
# PROCESSING WITH DIFFERENT FORMATS
# ============================================================

# 1. MARKDOWN FORMAT (Best for tables and structure)
# Step 1: Converting PDF to images...
# Converting 7 pages to images...
# Page 1 converted (DPI: 300)
# Page 2 converted (DPI: 300)
# Page 3 converted (DPI: 300)
# Page 4 converted (DPI: 300)
# Page 5 converted (DPI: 300)
# Page 6 converted (DPI: 300)
# Page 7 converted (DPI: 300)
# Step 2: Processing page 1/7...
# c:\Users\Busra\Melikoz\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
#   warnings.warn(warn_msg)
# Step 2: Processing page 2/7...
# Step 2: Processing page 3/7...
# Step 2: Processing page 4/7...
# Step 2: Processing page 5/7...
# Step 2: Processing page 6/7...
# Step 2: Processing page 7/7...
# Step 3: Creating embedding-ready chunks...
# Processing completed successfully!
# Structured Content Preview:
# --- Page 1 ---
# # Calmly Writer Online

# # 8/25/25, 5.19 PM

# # Introduction

# # 1

# # This privacy policy describes the personal data that Shopify processes (which may include

# # collecting, organizing, structuring, storing, using, or disclosing) when you use or access

# # Shopify Consumer Services, or when you interact with a merchant that is powered by

# # Shopify's platform ("Shopify Merchants" or "Merchants") . If any provisions in this policy
# ...
# Page 6 converted (DPI: 300)
# Page 7 converted (DPI: 300)
# Step 2: Processing page 1/7...
# Error: Object of type int32 is not JSON serializable
# Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
# "\nRECOMMENDED APPROACHES FOR EMBEDDINGS:\n\n1. **For Financial Documents (like credit reports):**\n   - Use Markdown format to preserve table structure\n   - Create chunks that respect logical boundaries (complete sections)\n   - Include metadata about content type (header, table, text)\n\n2. **Best Practices:**\n   - DPI 300+ for better text detection\n   - Structure-aware chunking (don't split tables)\n   - Preserve context with section headers\n   - Include bounding box information for spatial understanding\n\n3. **For Better Embeddings:**\n   - Keep related information together (account details + payment history)\n   - Use semantic boundaries rather than fixed character limits\n   - Include document structure markers\n   - Preserve table relationships\n\nInstallation:\npip install easyocr PyMuPDF pillow opencv-python pandas numpy\n"

In [ ]:
#3PDF to image
import fitz  # PyMuPDF for PDF -> images
import os
import base64
import json
import requests
import shutil

class PDFImageExtractor:
    def init(self, pdf_path, output_folder="pdf_images"):
        self.pdf_path = pdf_path
        self.output_folder = output_folder

    def pdf_to_images(self):
        """Convert each page of PDF to JPG image and return paths."""
        doc = fitz.open(self.pdf_path)
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

        image_paths = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=200)  # higher DPI for better quality
            image_path = os.path.join(self.outputfolder, f"page{page_num + 1}.jpg")
            pix.save(image_path)
            image_paths.append(image_path)
            print(f"✅ Page {page_num + 1} saved as {image_path}")
        doc.close()
        return image_paths

    def cleanup_images(self):
        """Delete all images from the output folder after extraction."""
        if os.path.exists(self.output_folder):
            shutil.rmtree(self.output_folder)
            print(f"🗑️ Deleted temporary folder: {self.output_folder}")

#gemma3:12b
class GemmaOCR:
    def init(self, model="gemma3:12b", ollama_url="http://localhost:11434/api/generate"):
        self.model = model
        self.ollama_url = ollama_url

    def encode_image(self, image_path):
        """Convert image to base64 string."""
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")

    def extract_text_from_image(self, image_path):
        """Send image to Gemma model for OCR-like extraction."""
        image_b64 = self.encode_image(image_path)

        payload = {
            "model": self.model,
            "prompt": "Extract and return all readable text from this image.",
            "images": [image_b64]
        }

        response = requests.post(self.ollama_url, json=payload, stream=True)
        extracted_text = ""

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode("utf-8"))
                    extracted_text += data.get("response", "")
                except Exception:
                    pass

        return extracted_text.strip()

def process_pdf_with_gemma(pdf_path):
    """Main pipeline: PDF -> Images -> Gemma OCR -> Combined text."""
    extractor = PDFImageExtractor(pdf_path)
    gemma_ocr = GemmaOCR()

    image_paths = extractor.pdf_to_images()
    all_text = ""

    for idx, image_path in enumerate(image_paths):
        print(f"\n🔍 Processing Page {idx + 1}...")
        page_text = gemma_ocr.extract_text_from_image(image_path)
        all_text += f"\n\n--- Page {idx + 1} ---\n{page_text}"

    # Clean up images after extraction
    extractor.cleanup_images()
    return all_text

if __name__ == "__main__":
    pdf_path = "cardinal.pdf"   # change to your PDF file path
    output_text = process_pdf_with_gemma(pdf_path)

    # Use PDF filename (without extension) for output text file
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    output_file = f"{base_name}.txt"

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(output_text)

    print(f"\n✅ Extraction Complete! Text saved to {output_file}")

In [ ]:
#qwen
import os
import json
import requests
import chromadb
from typing import List
from tqdm import tqdm

class TextChunker:
    def __init__(self, chunk_size: int, overlap: int):
        self.chunk_size = chunk_size
        self.overlap = overlap

    def chunk_text(self, text: str) -> List[str]:
        """Splits the text into chunks with overlap."""
        words = text.split()
        chunks = []
        
        for i in range(0, len(words), self.chunk_size - self.overlap):
            chunk = words[i:i + self.chunk_size]
            chunks.append(" ".join(chunk))
        
        return chunks


class EmbeddingProcessor:
    def __init__(self, model="dengcao/Qwen3-Embedding-8B:Q8_0", ollama_url="http://localhost:11434/api/generate"):
        self.model = model
        self.ollama_url = ollama_url

    def embed_text(self, text: str) -> List[float]:
        """Generate embeddings for the provided text using Ollama."""
        payload = {
            "model": self.model,
            "prompt": f"Embed the following text: {text}"
        }

        response = requests.post(self.ollama_url, json=payload)
        if response.status_code == 200:
            data = response.json()
            return data.get("embedding", [])
        else:
            raise Exception(f"Error generating embedding: {response.text}")


class ChromaDBHandler:
    def __init__(self, db_name="pdf_embeddings"):
        self.client = chromadb.Client()
        self.collection = self.client.create_collection(db_name)

    def store_embeddings(self, chunk_id: str, source: str, content_preview: str, embedding: List[float]):
        """Store embeddings in ChromaDB."""
        self.collection.add(
            documents=[content_preview],
            embeddings=[embedding],
            metadatas=[{"source": source}],
            ids=[chunk_id]
        )

    def query_embeddings(self, query_embedding: List[float], n_results=3):
        """Query the ChromaDB collection for similar embeddings."""
        results = self.collection.query(query_embeddings=query_embedding, n_results=n_results)
        return results


def process_pdf_embeddings(pdf_text_path, chunk_size=300, overlap=30):
    # Read the text from the extracted file
    with open(pdf_text_path, "r", encoding="utf-8") as f:
        pdf_text = f.read()

    # Split the text into chunks
    chunker = TextChunker(chunk_size=chunk_size, overlap=overlap)
    chunks = chunker.chunk_text(pdf_text)

    # Set up the Embedding processor and ChromaDB handler
    embedding_processor = EmbeddingProcessor()
    chromadb_handler = ChromaDBHandler()

    # Embedding the chunks and storing in ChromaDB
    all_embeddings = []
    chunk_ids = []
    for idx, chunk in enumerate(tqdm(chunks, desc="Embedding chunks")):
        chunk_id = f"chunk_{idx + 1}"
        embedding = embedding_processor.embed_text(chunk)
        content_preview = chunk[:100]  # Get first 100 characters for preview
        chromadb_handler.store_embeddings(chunk_id, pdf_text_path, content_preview, embedding)

        all_embeddings.append(embedding)
        chunk_ids.append(chunk_id)

    print(f"\n✅ Successfully embedded {len(chunks)} chunks and stored in ChromaDB collection.")

    return chunk_ids


def query_similar_embeddings(query_text, chunk_size=300, overlap=30):
    # Generate the query embedding
    chunker = TextChunker(chunk_size=chunk_size, overlap=overlap)
    chunks = chunker.chunk_text(query_text)

    embedding_processor = EmbeddingProcessor()
    query_embedding = embedding_processor.embed_text(" ".join(chunks))

    # Set up ChromaDB handler for querying
    chromadb_handler = ChromaDBHandler()
    results = chromadb_handler.query_embeddings(query_embedding)

    # Print query results
    print(f"\n🔍 Query Results:")
    for idx, result in enumerate(results["documents"]):
        print(f"\n🔹 ID: {results['ids'][idx]}")
        print(f"📄 Source: {result['metadata']['source']}")
        print(f"📝 Content Preview: {result['document'][:200]}")
        print(f"📊 Distance: {results['distances'][idx]}")
        print("-" * 80)


if __name__ == "__main__":
    # Assuming that the text file is generated already
    pdf_text_path = "privacy_policy.txt"  # Path to the text file with extracted text

    # Process the PDF text and store embeddings
    chunk_ids = process_pdf_embeddings(pdf_text_path)

    # Example of querying the stored embeddings
    query_text = "What is the return policy for electronics?"  # Example query
    query_similar_embeddings(query_text)


## chatbot implement

In [ ]:
import ollama

# Function to initialize and interact with the chatbot
def chat():
    print("Welcome to the chatbot! Type 'exit' to end the conversation.")
    
    history = []  
    
    while True:
        
        user_input = input("You: ")
        
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        
        history.append({"role": "user", "content": user_input})

        
        response = ollama.chat(model="qwen3:8b", messages=history)
        
        # Print the whole response to inspect its structure
        print("Full Response:", response)
        
        # Extract the assistant's response from the message
        try:
            chatbot_reply = response.message.content if response.message else "No reply"
            print(f"Bot: {chatbot_reply}")
        except AttributeError as e:
            print(f"Error: Missing expected attribute {e} in the response.")
        
        # Add the assistant's response to history
        history.append({"role": "assistant", "content": chatbot_reply})

# Run the chatbot function
if __name__ == "__main__":
    chat()

#worked but need time


Welcome to the chatbot! Type 'exit' to end the conversation.
Full Response: model='qwen3:8b' created_at='2025-09-17T05:35:09.5152139Z' done=True done_reason='stop' total_duration=25776657600 load_duration=46179800 prompt_eval_count=9 prompt_eval_duration=323154100 eval_count=106 eval_duration=25406821400 message=Message(role='assistant', content='<think>\nOkay, the user said "hello". I need to respond in a friendly and helpful manner. Let me start by acknowledging their greeting. Maybe say something like "Hello!" to be polite. Then, I should ask how I can assist them. That way, it opens up the conversation for them to ask questions or request help. Keep it simple and welcoming. Let me make sure the tone is positive and approachable. Alright, that should work.\n</think>\n\nHello! 😊 How can I assist you today?', thinking=None, images=None, tool_name=None, tool_calls=None)
Bot: <think>
Okay, the user said "hello". I need to respond in a friendly and helpful manner. Let me start by acknowl

In [ ]:
#demo 2 chatbot
import ollama

def chat_with_bot():
    # Define your system prompt here
    system_prompt = "You are a helpful assistant that always replies in a friendly and concise way."

    print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")

    # Store conversation history
    conversation = [
        {"role": "system", "content": system_prompt}
    ]

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("🤖 Bot: Goodbye!")
            break

        # Add user message to conversation
        conversation.append({"role": "user", "content": user_input})

        # Send conversation to Ollama
        response = ollama.chat(
            model="qwen3:8b",
            messages=conversation
            # temperature=0.7,
            # max_token=512,
            # top_p=0.9,
            # frequency_penalty=0,
            # presence_penalty=0.6,
            # stop=None,
            # stream=False,
            # n=1,
        )

        bot_reply = response["message"]["content"]
        print("🤖 Bot:", bot_reply)

        # Save bot reply in conversation history
        conversation.append({"role": "assistant", "content": bot_reply})

if __name__ == "__main__":
    chat_with_bot()
 

 #

🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.

🤖 Bot: <think>
Okay, the user said "hello". I need to respond in a friendly and concise way. Let me make sure to greet them back and offer help. Maybe something like, "Hello! How can I assist you today?" That's straightforward and opens the door for them to ask anything they need. I should keep it simple and positive.
</think>

Hello! How can I assist you today? 😊
🤖 Bot: <think>
Okay, the user asked for a short explanation about a full moon. Let me recall what I know. A full moon occurs when the Moon is on the opposite side of Earth from the Sun, so the Sun's light reflects off the Moon's surface and is visible from Earth. This happens once every lunar month, which is about 29.5 days. During a full moon, the Moon appears fully illuminated. I should mention the phases leading up to it, like the waning gibbous and then the full moon. Also, maybe note that it's when the Moon is farthest from Earth in its orbit, but wait, no, the full mo

In [ ]:
# import ollama
# import chromadb
# from chromadb.config import Settings

# class KnowledgeBase:
#     def __init__(self, db_path="./chroma_db"):
#         # Initialize ChromaDB client
#         self.client = chromadb.PersistentClient(path=db_path)
#         self.collection_name = "pdf_embeddings"
        
#         # Create or get collection
#         self.collection = self.client.get_or_create_collection(
#             name=self.collection_name,
#             metadata={"description": "PDF text embeddings"}
#         )

#     def search_similar(self, query, n_results=3):
#         """Search for similar chunks in the ChromaDB collection."""
#         results = self.collection.query(
#             query_texts=[query],
#             n_results=n_results
#         )
#         return results


# def chat_with_bot():
#     # Initialize ChromaDB knowledge base
#     kb = KnowledgeBase()

#     # Define your system prompt
#     system_prompt = "You are a helpful assistant that always replies in a friendly and concise way."

#     print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")

#     # Store conversation history
#     conversation = [
#         {"role": "system", "content": system_prompt}
#     ]

#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ["exit", "quit", "bye"]:
#             print("🤖 Bot: Goodbye!")
#             break

#         # Add user message to conversation
#         conversation.append({"role": "user", "content": user_input})

#         # Search for relevant information from the knowledge base
#         kb_results = kb.search_similar(user_input)

#         # Combine the knowledge base results into the conversation
#         kb_response = ""
        
#         # Process and include top results from ChromaDB
#         for idx, document in enumerate(kb_results['documents'][0]):
#             kb_response += f"📄 Chunk {idx+1}: {document[:300]}...\n"
        
#         # If there are results, include them in the conversation
#         if kb_response:
#             knowledge_base_message = f"Here is some relevant information I found from the knowledge base:\n{kb_response}"
#         else:
#             knowledge_base_message = "I couldn't find anything relevant in my knowledge base."

#         # Add KB response to conversation
#         conversation.append({"role": "assistant", "content": knowledge_base_message})

#         # Send conversation to Ollama for further processing and response
#         response = ollama.chat(
#             model="qwen3:8b",
#             messages=conversation
#         )

#         bot_reply = response["message"]["content"]
#         print("🤖 Bot:", bot_reply)

#         # Save bot reply in conversation history
#         conversation.append({"role": "assistant", "content": bot_reply})


# if __name__ == "__main__":
#     chat_with_bot()


import ollama
import chromadb
from chromadb.config import Settings


class KnowledgeBase:
    def __init__(self, db_path="./chroma_db"):
        # Initialize ChromaDB client
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        
        # Create or get collection
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )

    def search_similar(self, query, n_results=3):
        """Search for similar chunks in the ChromaDB collection."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        
        # Return documents and their metadata separately
        documents = results['documents'][0]
        metadatas = results['metadatas'][0]
        return documents, metadatas

def chat_with_bot():
    # Initialize ChromaDB knowledge base
    kb = KnowledgeBase()

    # Define your system prompt
    system_prompt = """
    You are a helpful assistant designed to assist with specific topics. 
    Your responses should be informative, concise, and polite. 
    You have access to a knowledge base that contains information from PDF documents, which can help provide accurate answers to user queries.
    If the user asks for help related to the content in the PDFs, retrieve the most relevant chunks and incorporate them into your responses.
    Always aim to be clear and direct in your explanations, and provide examples or context where possible.
    """

    print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")

    # Store conversation history
    conversation = [
        {"role": "system", "content": system_prompt}
    ]

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("🤖 Bot: Goodbye!")
            break

        # Add user message to conversation
        conversation.append({"role": "user", "content": user_input})

        # Search for relevant information from the knowledge base
        kb_documents, kb_metadatas = kb.search_similar(user_input)

        # Combine the knowledge base results into the conversation
        kb_response = ""
        
        # Process and include top results from ChromaDB
        if kb_documents:
            kb_response = "Here is some relevant information I found from the knowledge base:\n"
            for idx, document in enumerate(kb_documents):
                kb_response += f"📄 Chunk {idx+1}: {document[:300]}...\n"
        
        if not kb_documents:
            kb_response = "I couldn't find anything relevant in my knowledge base."
        
        # Add KB response to conversation
        conversation.append({"role": "assistant", "content": kb_response})

        # Send conversation to Ollama for further processing and response
        response = ollama.chat(
            model="qwen3:8b",
            messages=conversation
        )

        bot_reply = response["message"]["content"]
        print("🤖 Bot:", bot_reply)

        # Save bot reply in conversation history
        conversation.append({"role": "assistant", "content": bot_reply})

if __name__ == "__main__":
    chat_with_bot()



🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.

🤖 Bot: 📄 Chunk 4: you can opt out of targeted advertising through your browser settings, or through your device settings, such as on an iPhone or Android phone. You can also opt out of targeted advertising by using the opt-out links provided by the advertising network. ...
📄 Chunk 5: You can opt out of interest-based ads by visiting the Network Advertising Initiative's opt-out page. This will opt you out of receiving targeted ads from many of the major ad networks. ...

Based on the information provided, here's the answer to your question:

Yes, you can control the type of advertisements you see or choose to opt out entirely. Here are the methods you can use:

1. **Browser or Device Settings**: You can opt out of targeted advertising through your browser settings or device settings (such as on an iPhone or Android phone). These settings allow you to manage your ad preferences and limit the types of ads you see.

2. **Advertising Netw

## merge code all

In [ ]:
# import easyocr
# import fitz  # PyMuPDF
# import chromadb
# import ollama
# import os
# from PIL import Image
# import numpy as np
# import cv2
# from typing import List, Tuple
# from chromadb.config import Settings

# # EasyOCR and PDF Processing Class
# class PDFOCRProcessor:
#     def __init__(self, languages=['en'], gpu=False):
#         print("Initializing EasyOCR...")
#         self.reader = easyocr.Reader(languages, gpu=gpu)
#         print("EasyOCR initialized successfully!")

#     def pdf_to_images(self, pdf_path: str, output_dir: str = None, dpi: int = 200) -> List[str]:
#         if not os.path.exists(pdf_path):
#             raise FileNotFoundError(f"PDF file not found: {pdf_path}")
#         if output_dir:
#             os.makedirs(output_dir, exist_ok=True)
#         doc = fitz.open(pdf_path)
#         image_paths = []
#         print(f"Converting {len(doc)} pages to images...")
#         for page_num in range(len(doc)):
#             page = doc[page_num]
#             mat = fitz.Matrix(dpi/72, dpi/72)
#             pix = page.get_pixmap(matrix=mat)
#             img_data = pix.tobytes("png")
#             img = Image.open(io.BytesIO(img_data)) if 'io' in globals() else None
#             if img is None:
#                 img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
#             if output_dir:
#                 image_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
#                 img.save(image_path, "PNG")
#                 image_paths.append(image_path)
#             else:
#                 image_path = f"temp_page_{page_num + 1}.png"
#                 img.save(image_path, "PNG")
#                 image_paths.append(image_path)
#             print(f"Page {page_num + 1} converted to image")
#         doc.close()
#         return image_paths

#     def enhance_image_for_ocr(self, image_path: str) -> np.ndarray:
#         img = cv2.imread(image_path)
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         denoised = cv2.fastNlMeansDenoising(gray)
#         thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
#         return thresh

#     def extract_text_from_image(self, image_path: str, enhance: bool = True) -> List[Tuple[str, float]]:
#         if enhance:
#             img = self.enhance_image_for_ocr(image_path)
#             results = self.reader.readtext(img)
#         else:
#             results = self.reader.readtext(image_path)
#         text_results = [(result[1], result[2]) for result in results]
#         return text_results

#     def process_pdf(self, pdf_path: str, output_dir: str = None, enhance_images: bool = True, confidence_threshold: float = 0.5) -> dict:
#         results = {'pdf_path': pdf_path, 'pages': [], 'total_pages': 0, 'extracted_text': ''}
#         try:
#             print("Step 1: Converting PDF to images...")
#             image_paths = self.pdf_to_images(pdf_path, output_dir)
#             results['total_pages'] = len(image_paths)
#             print("Step 2: Extracting text from images...")
#             all_text = []
#             for i, image_path in enumerate(image_paths):
#                 print(f"Processing page {i + 1}/{len(image_paths)}...")
#                 text_results = self.extract_text_from_image(image_path, enhance_images)
#                 filtered_text = [text for text, conf in text_results if conf >= confidence_threshold]
#                 page_text = ' '.join(filtered_text)
#                 all_text.append(page_text)
#                 page_info = {'page_number': i + 1, 'image_path': image_path, 'text': page_text}
#                 results['pages'].append(page_info)
#             results['extracted_text'] = '\n\n--- Page Break ---\n\n'.join(all_text)
#         except Exception as e:
#             results['error'] = str(e)
#         return results


# # ChromaDB Embedding Class
# class TextEmbedder:
#     def __init__(self, db_path="./Merge_chroma_db"):
#         self.client = chromadb.PersistentClient(path=db_path)
#         self.collection_name = "pdf_embeddings"
#         self.collection = self.client.get_or_create_collection(
#             name=self.collection_name,
#             metadata={"description": "PDF text embeddings"}
#         )

#     def chunk_text(self, text, chunk_size=500, overlap=50):
#         chunks = []
#         words = text.split()
#         for i in range(0, len(words), chunk_size - overlap):
#             chunk = " ".join(words[i:i + chunk_size])
#             if chunk.strip():
#                 chunks.append(chunk.strip())
#         return chunks

#     def embed_text_file(self, txt_file_path):
#         if not os.path.exists(txt_file_path):
#             print(f"❌ File not found: {txt_file_path}")
#             return
#         with open(txt_file_path, "r", encoding="utf-8") as f:
#             text = f.read()
#         chunks = self.chunk_text(text)
#         documents = chunks
#         ids = [f"chunk_{i+1}" for i in range(len(chunks))]
#         metadatas = [{"source": txt_file_path, "chunk_index": i+1} for i in range(len(chunks))]
#         self.collection.add(documents=documents, ids=ids, metadatas=metadatas)
#         print(f"✅ Successfully embedded {len(chunks)} chunks from {txt_file_path}")

#     def search_similar(self, query, n_results=3):
#         results = self.collection.query(
#             query_texts=[query],
#             n_results=n_results
#         )
#         return results['documents'][0]


# # Chatbot using Ollama and ChromaDB
# class KnowledgeBase:
#     def __init__(self, db_path="./Merge_chroma_db"):
#         self.client = chromadb.PersistentClient(path=db_path)
#         self.collection_name = "pdf_embeddings"
#         self.collection = self.client.get_or_create_collection(
#             name=self.collection_name,
#             metadata={"description": "PDF text embeddings"}
#         )

#     def search_similar(self, query, n_results=3):
#         results = self.collection.query(
#             query_texts=[query],
#             n_results=n_results
#         )
#         documents = results['documents'][0]
#         metadatas = results['metadatas'][0]
#         return documents, metadatas


# def chat_with_bot():
#     kb = KnowledgeBase()
#     system_prompt = """
#     You are a helpful assistant designed to assist with specific topics. 
#     Your responses should be informative, concise, and polite. 
#     You have access to a knowledge base that contains information from PDF documents, which can help provide accurate answers to user queries.
#     If the user asks for help related to the content in the PDFs, retrieve the most relevant chunks and incorporate them into your responses.
#     Always aim to be clear and direct in your explanations, and provide examples or context where possible.
#     """

#     print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")
#     conversation = [{"role": "system", "content": system_prompt}]
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() in ["exit", "quit", "bye"]:
#             print("🤖 Bot: Goodbye!")
#             break
#         conversation.append({"role": "user", "content": user_input})
#         kb_documents, kb_metadatas = kb.search_similar(user_input)
#         kb_response = ""
#         if kb_documents:
#             kb_response = "Here is some relevant information I found from the knowledge base:\n"
#             for idx, document in enumerate(kb_documents):
#                 kb_response += f"📄 Chunk {idx+1}: {document[:300]}...\n"
#         if not kb_documents:
#             kb_response = "I couldn't find anything relevant in my knowledge base."
#         conversation.append({"role": "assistant", "content": kb_response})
#         response = ollama.chat(model="qwen3:8b", messages=conversation)
#         bot_reply = response["message"]["content"]
#         print("🤖 Bot:", bot_reply)
#         conversation.append({"role": "assistant", "content": bot_reply})


# def main():
#     # Step 1: Extract text from PDF
#     pdf_path = "./New folder/privacy_policy.pdf"  # Change this to your PDF path
#     processor = PDFOCRProcessor(languages=['en'], gpu=False)
#     results = processor.process_pdf(pdf_path, output_dir="extracted_images", enhance_images=True, confidence_threshold=0.5)

#     if 'error' not in results:
#         # Step 2: Save the extracted text to a file
#         with open('merge__extracted_text.txt', 'w', encoding='utf-8') as f:
#             f.write(results['extracted_text'])
#         print(f"Extracted text saved to: merge__extracted_text.txt")

#         # Step 3: Embed the text file into ChromaDB
#         embedder = TextEmbedder()
#         embedder.embed_text_file("merge__extracted_text.txt")

#         # Step 4: Start the Chatbot
#         chat_with_bot()
#     else:
#         print(f"Error occurred: {results['error']}")

# if __name__ == "__main__":
#     main()
###

import easyocr
import ollama
import fitz  # PyMuPDF for PDF -> images
import os
import shutil
from typing import List, Tuple
import cv2
from PIL import Image
import numpy as np
import chromadb
from typing import List, Tuple
from chromadb.config import Settings

class PDFOCRProcessor:
    """PDF to Image and OCR Processor with cleanup of temporary image folder."""

    def __init__(self, languages=['en'], gpu=False, pdf_path=None, output_folder="pdf_images"):
        """
        Initialize the OCR Processor.
        
        Args:
            languages (list): List of languages for OCR (e.g., ['en', 'es'])
            gpu (bool): Whether to use GPU for OCR processing
            pdf_path (str): Path to the PDF file to process
            output_folder (str): Folder to save images temporarily
        """
        print("Initializing EasyOCR...")
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("EasyOCR initialized successfully!")
        self.pdf_path = pdf_path
        self.output_folder = output_folder

    def pdf_to_images(self):
        """Convert each page of PDF to images and return paths."""
        if not os.path.exists(self.pdf_path):
            raise FileNotFoundError(f"PDF file not found: {self.pdf_path}")
        
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

        doc = fitz.open(self.pdf_path)
        image_paths = []
        print(f"Converting {len(doc)} pages to images...")
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=200)  # Higher DPI for better OCR quality
            image_path = os.path.join(self.output_folder, f"page_{page_num + 1}.png")
            pix.save(image_path)
            image_paths.append(image_path)
            print(f"✅ Page {page_num + 1} saved as {image_path}")
        doc.close()
        return image_paths

    def enhance_image_for_ocr(self, image_path: str) -> np.ndarray:
        """Enhance image quality for better OCR results."""
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        return thresh

    def extract_text_from_image(self, image_path: str, enhance: bool = True) -> List[Tuple[str, float]]:
        """Extract text from image using EasyOCR."""
        if enhance:
            img = self.enhance_image_for_ocr(image_path)
            results = self.reader.readtext(img)
        else:
            results = self.reader.readtext(image_path)
        text_results = [(result[1], result[2]) for result in results]
        return text_results

    def cleanup_images(self):
        """Delete all images from the output folder after extraction."""
        if os.path.exists(self.output_folder):
            shutil.rmtree(self.output_folder)
            print(f"🗑️ Deleted temporary folder: {self.output_folder}")

    def process_pdf(self):
        """Full pipeline: PDF -> images -> OCR -> Text extraction."""
        results = {'pdf_path': self.pdf_path, 'pages': [], 'total_pages': 0, 'extracted_text': ''}
        try:
            print("Step 1: Converting PDF to images...")
            image_paths = self.pdf_to_images()
            results['total_pages'] = len(image_paths)
            print("Step 2: Extracting text from images...")
            all_text = []
            for i, image_path in enumerate(image_paths):
                print(f"Processing page {i + 1}/{len(image_paths)}...")
                text_results = self.extract_text_from_image(image_path, enhance=True)
                page_text = ' '.join([text for text, conf in text_results if conf >= 0.5])
                all_text.append(page_text)
                page_info = {'page_number': i + 1, 'image_path': image_path, 'text': page_text}
                results['pages'].append(page_info)
            results['extracted_text'] = '\n\n--- Page Break ---\n\n'.join(all_text)

            # Clean up temporary images after OCR extraction
            self.cleanup_images()

        except Exception as e:
            results['error'] = str(e)

        return results


class TextEmbedder:
    """Text Embedder for storing extracted text in ChromaDB."""

    def __init__(self, db_path="./merge__chroma_db"):
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )

    def chunk_text(self, text, chunk_size=500, overlap=50):
        """Split text into overlapping chunks."""
        chunks = []
        words = text.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk.strip())
        return chunks

    def embed_text_file(self, txt_file_path):
        """Read text file and embed it chunk by chunk."""
        if not os.path.exists(txt_file_path):
            print(f"❌ File not found: {txt_file_path}")
            return

        with open(txt_file_path, "r", encoding="utf-8") as f:
            text = f.read()
        chunks = self.chunk_text(text)
        documents = chunks
        ids = [f"chunk_{i+1}" for i in range(len(chunks))]
        metadatas = [{"source": txt_file_path, "chunk_index": i+1} for i in range(len(chunks))]
        self.collection.add(documents=documents, ids=ids, metadatas=metadatas)
        print(f"✅ Successfully embedded {len(chunks)} chunks from {txt_file_path}")

    def search_similar(self, query, n_results=3):
        """Search for similar chunks."""
        results = self.collection.query(query_texts=[query], n_results=n_results)
        return results['documents'][0]


class KnowledgeBase:
    """Knowledge base class to interact with ChromaDB for search."""

    def __init__(self, db_path="./merge__chroma_db"):
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )

    def search_similar(self, query, n_results=3):
        """Search for similar chunks in the ChromaDB collection."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        documents = results['documents'][0]
        metadatas = results['metadatas'][0]
        return documents, metadatas


def chat_with_bot():
    kb = KnowledgeBase()
    system_prompt = """
    You are a helpful assistant designed to assist with specific topics. 
    Your responses should be informative, concise, and polite. 
    You have access to a knowledge base that contains information from PDF documents, which can help provide accurate answers to user queries.
    If the user asks for help related to the content in the PDFs, retrieve the most relevant chunks and incorporate them into your responses.
    Always aim to be clear and direct in your explanations, and provide examples or context where possible.
    """

    print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")
    conversation = [{"role": "system", "content": system_prompt}]
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("🤖 Bot: Goodbye!")
            break
        conversation.append({"role": "user", "content": user_input})
        kb_documents, kb_metadatas = kb.search_similar(user_input)
        kb_response = ""
        if kb_documents:
            kb_response = "Here is some relevant information I found from the knowledge base:\n"
            for idx, document in enumerate(kb_documents):
                kb_response += f"📄 Chunk {idx+1}: {document[:300]}...\n"
        if not kb_documents:
            kb_response = "I couldn't find anything relevant in my knowledge base."
        conversation.append({"role": "assistant", "content": kb_response})
        response = ollama.chat(model="qwen3:8b", messages=conversation)
        bot_reply = response["message"]["content"]
        print("🤖 Bot:", bot_reply)
        conversation.append({"role": "assistant", "content": bot_reply})


def main():
    # Step 1: Extract text from PDF
    pdf_path = "./New folder/privacy_policy.pdf"  # Change this to your PDF path
    processor = PDFOCRProcessor(pdf_path=pdf_path, languages=['en'], gpu=False, output_folder="pdf_images")
    results = processor.process_pdf()

    if 'error' not in results:
        # Step 2: Save the extracted text to a file
        with open('merge__extracted_text.txt', 'w', encoding='utf-8') as f:
            f.write(results['extracted_text'])
        print(f"Extracted text saved to: merge__extracted_text.txt")

        # Step 3: Embed the text file into ChromaDB
        embedder = TextEmbedder()
        embedder.embed_text_file("merge__extracted_text.txt")

        # Step 4: Start the Chatbot
        chat_with_bot()
    else:
        print(f"Error occurred: {results['error']}")

if __name__ == "__main__":
    main()



Using CPU. Note: This module is much faster with a GPU.


Initializing EasyOCR...
EasyOCR initialized successfully!
Step 1: Converting PDF to images...
Converting 7 pages to images...
✅ Page 1 saved as pdf_images\page_1.png
✅ Page 2 saved as pdf_images\page_2.png
✅ Page 3 saved as pdf_images\page_3.png
✅ Page 4 saved as pdf_images\page_4.png
✅ Page 5 saved as pdf_images\page_5.png
✅ Page 6 saved as pdf_images\page_6.png
✅ Page 7 saved as pdf_images\page_7.png
Step 2: Extracting text from images...
Processing page 1/7...


c:\Users\Busra\Melikoz\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Processing page 2/7...
Processing page 3/7...
Processing page 4/7...
Processing page 5/7...
Processing page 6/7...
Processing page 7/7...
🗑️ Deleted temporary folder: pdf_images
Extracted text saved to: merge__extracted_text.txt
✅ Successfully embedded 5 chunks from merge__extracted_text.txt
🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.

🤖 Bot: 📄 Chunk 4: We may share your personal data with third parties for the purposes of providing our services or services offered by our Merchants, including: (1) our service providers who perform services on our behalf, such as hosting, payment processing, customer service, and marketing; (2) our parent company, subsidiaries, and affiliates, including Shopify, Shopify Payments, and Shopify Plus; (3) merchants who are part of the Shopify platform, including those who sell products or services on the Shopify platform; (4) third parties that provide products or services that integrate with Shopify, such...
📄 Chunk 5: Shopify's privacy policy is d

## multiple pdf handeling

In [ ]:
import easyocr
import fitz  # PyMuPDF for PDF -> images
import os
import shutil
import ollama
import cv2
import numpy as np
from typing import List, Tuple
from PIL import Image
import chromadb
from chromadb.config import Settings



class PDFOCRProcessor:
    """PDF to Image and OCR Processor with cleanup of temporary image folder."""

    def __init__(self, languages=['en'], gpu=False, pdf_path=None, output_folder="pdf_images"):
        """
        Initialize the OCR Processor.
        
        Args:
            languages (list): List of languages for OCR (e.g., ['en', 'es'])
            gpu (bool): Whether to use GPU for OCR processing
            pdf_path (str): Path to the PDF file to process
            output_folder (str): Folder to save images temporarily
        """
        print("Initializing EasyOCR...")
        self.reader = easyocr.Reader(languages, gpu=gpu)
        print("EasyOCR initialized successfully!")
        self.pdf_path = pdf_path
        self.output_folder = output_folder

    def pdf_to_images(self):
        """Convert each page of PDF to images and return paths."""
        if not os.path.exists(self.pdf_path):
            raise FileNotFoundError(f"PDF file not found: {self.pdf_path}")
        
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)

        doc = fitz.open(self.pdf_path)
        image_paths = []
        print(f"Converting {len(doc)} pages to images...")
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=200)  # Higher DPI for better OCR quality
            image_path = os.path.join(self.output_folder, f"page_{page_num + 1}.png")
            pix.save(image_path)
            image_paths.append(image_path)
            print(f"✅ Page {page_num + 1} saved as {image_path}")
        doc.close()
        return image_paths

    def enhance_image_for_ocr(self, image_path: str) -> np.ndarray:
        """Enhance image quality for better OCR results."""
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        denoised = cv2.fastNlMeansDenoising(gray)
        thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        return thresh

    def extract_text_from_image(self, image_path: str, enhance: bool = True) -> List[Tuple[str, float]]:
        """Extract text from image using EasyOCR."""
        if enhance:
            img = self.enhance_image_for_ocr(image_path)
            results = self.reader.readtext(img)
        else:
            results = self.reader.readtext(image_path)
        text_results = [(result[1], result[2]) for result in results]
        return text_results

    def cleanup_images(self):
        """Delete all images from the output folder after extraction."""
        if os.path.exists(self.output_folder):
            shutil.rmtree(self.output_folder)
            print(f"🗑️ Deleted temporary folder: {self.output_folder}")

    def process_pdf(self):
        """Full pipeline: PDF -> images -> OCR -> Text extraction."""
        results = {'pdf_path': self.pdf_path, 'pages': [], 'total_pages': 0, 'extracted_text': ''}
        try:
            print("Step 1: Converting PDF to images...")
            image_paths = self.pdf_to_images()
            results['total_pages'] = len(image_paths)
            print("Step 2: Extracting text from images...")
            all_text = []
            for i, image_path in enumerate(image_paths):
                print(f"Processing page {i + 1}/{len(image_paths)}...")
                text_results = self.extract_text_from_image(image_path, enhance=True)
                page_text = ' '.join([text for text, conf in text_results if conf >= 0.5])
                all_text.append(page_text)
                page_info = {'page_number': i + 1, 'image_path': image_path, 'text': page_text}
                results['pages'].append(page_info)
            results['extracted_text'] = '\n\n--- Page Break ---\n\n'.join(all_text)

            # Clean up temporary images after OCR extraction
            self.cleanup_images()

        except Exception as e:
            results['error'] = str(e)

        return results


class TextEmbedder:
    """Text Embedder for storing extracted text in ChromaDB."""

    def __init__(self, db_path="./multipdf_merge_chroma_db"):
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )

    def chunk_text(self, text, chunk_size=500, overlap=50):
        """Split text into overlapping chunks."""
        chunks = []
        words = text.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            if chunk.strip():
                chunks.append(chunk.strip())
        return chunks

    def embed_text_file(self, txt_file_path):
        """Read text file and embed it chunk by chunk."""
        if not os.path.exists(txt_file_path):
            print(f"❌ File not found: {txt_file_path}")
            return

        with open(txt_file_path, "r", encoding="utf-8") as f:
            text = f.read()
        chunks = self.chunk_text(text)
        documents = chunks
        ids = [f"chunk_{i+1}" for i in range(len(chunks))]
        metadatas = [{"source": txt_file_path, "chunk_index": i+1} for i in range(len(chunks))]
        self.collection.add(documents=documents, ids=ids, metadatas=metadatas)
        print(f"✅ Successfully embedded {len(chunks)} chunks from {txt_file_path}")

    def search_similar(self, query, n_results=3):
        """Search for similar chunks."""
        results = self.collection.query(query_texts=[query], n_results=n_results)
        return results['documents'][0]


class KnowledgeBase:
    """Knowledge base class to interact with ChromaDB for search."""

    def __init__(self, db_path="./multipdf_merge_chroma_db"):
        self.client = chromadb.PersistentClient(path=db_path)
        self.collection_name = "pdf_embeddings"
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF text embeddings"}
        )

    def search_similar(self, query, n_results=3):
        """Search for similar chunks in the ChromaDB collection."""
        results = self.collection.query(
            query_texts=[query],
            n_results=n_results
        )
        documents = results['documents'][0]
        metadatas = results['metadatas'][0]
        return documents, metadatas


def chat_with_bot():
    kb = KnowledgeBase()
    system_prompt = """
    You are a helpful assistant designed to assist with specific topics. 
    Your responses should be informative, concise, and polite. 
    You have access to a knowledge base that contains information from PDF documents, which can help provide accurate answers to user queries.
    If the user asks for help related to the content in the PDFs, retrieve the most relevant chunks and incorporate them into your responses.
    Always aim to be clear and direct in your explanations, and provide examples or context where possible.
    """

    print("🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.\n")
    conversation = [{"role": "system", "content": system_prompt}]
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("🤖 Bot: Goodbye!")
            break
        conversation.append({"role": "user", "content": user_input})
        kb_documents, kb_metadatas = kb.search_similar(user_input)
        kb_response = ""
        if kb_documents:
            kb_response = "Here is some relevant information I found from the knowledge base:\n"
            for idx, document in enumerate(kb_documents):
                kb_response += f"📄 Chunk {idx+1}: {document[:300]}...\n"
        if not kb_documents:
            kb_response = "I couldn't find anything relevant in my knowledge base."
        conversation.append({"role": "assistant", "content": kb_response})
        response = ollama.chat(model="qwen3:8b", messages=conversation)
        bot_reply = response["message"]["content"]
        print("🤖 Bot:", bot_reply)
        conversation.append({"role": "assistant", "content": bot_reply})


def process_multiple_pdfs_in_folder(pdf_folder_path):
    """Process all PDFs in the specified folder."""
    all_extracted_text = ""
    
    # Loop through all PDF files in the folder
    for pdf_file in os.listdir(pdf_folder_path):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, pdf_file)
            print(f"\nProcessing PDF: {pdf_path}")
            
            
            processor = PDFOCRProcessor(pdf_path=pdf_path, languages=['en'], gpu=False, output_folder="pdf_images")
            results = processor.process_pdf()

            if 'error' not in results:
                
                all_extracted_text += f"\n\n--- Extracted Text from {pdf_file} ---\n\n"
                all_extracted_text += results['extracted_text']
                print(f"Extracted text from {pdf_file} added.")

            else:
                print(f"Error occurred while processing {pdf_file}: {results['error']}")

    # Save the combined extracted text to a file
    with open('multipdf_merge_combined_extracted_text.txt', 'w', encoding='utf-8') as f:
        f.write(all_extracted_text)
    print(f"\n✅ Extraction Complete! All text saved to multipdf_merge_combined_extracted_text.txt")

    return 'multipdf_merge_combined_extracted_text.txt'


def main():
    pdf_folder_path = "./multi_pdf_test"  # Change this to your folder path with PDFs
    
    # Step 1: Process all PDFs in the folder and extract text
    combined_txt_file = process_multiple_pdfs_in_folder(pdf_folder_path)

    # Step 2: Embed the combined extracted text file into ChromaDB
    embedder = TextEmbedder()
    embedder.embed_text_file(combined_txt_file)

    # Step 3: Start the Chatbot
    chat_with_bot()


if __name__ == "__main__":
    main()



Using CPU. Note: This module is much faster with a GPU.



Processing PDF: ./multi_pdf_test\about_us.pdf
Initializing EasyOCR...
EasyOCR initialized successfully!
Step 1: Converting PDF to images...
Converting 4 pages to images...
✅ Page 1 saved as pdf_images\page_1.png
✅ Page 2 saved as pdf_images\page_2.png
✅ Page 3 saved as pdf_images\page_3.png
✅ Page 4 saved as pdf_images\page_4.png
Step 2: Extracting text from images...
Processing page 1/4...


c:\Users\Busra\Melikoz\venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Processing page 2/4...
Processing page 3/4...
Processing page 4/4...


Using CPU. Note: This module is much faster with a GPU.


🗑️ Deleted temporary folder: pdf_images
Extracted text from about_us.pdf added.

Processing PDF: ./multi_pdf_test\privacy_policy.pdf
Initializing EasyOCR...
EasyOCR initialized successfully!
Step 1: Converting PDF to images...
Converting 7 pages to images...
✅ Page 1 saved as pdf_images\page_1.png
✅ Page 2 saved as pdf_images\page_2.png
✅ Page 3 saved as pdf_images\page_3.png
✅ Page 4 saved as pdf_images\page_4.png
✅ Page 5 saved as pdf_images\page_5.png
✅ Page 6 saved as pdf_images\page_6.png
✅ Page 7 saved as pdf_images\page_7.png
Step 2: Extracting text from images...
Processing page 1/7...
Processing page 2/7...
Processing page 3/7...
Processing page 4/7...
Processing page 5/7...
Processing page 6/7...
Processing page 7/7...


Using CPU. Note: This module is much faster with a GPU.


🗑️ Deleted temporary folder: pdf_images
Extracted text from privacy_policy.pdf added.

Processing PDF: ./multi_pdf_test\Return Policy.pdf
Initializing EasyOCR...
EasyOCR initialized successfully!
Step 1: Converting PDF to images...
Converting 5 pages to images...
✅ Page 1 saved as pdf_images\page_1.png
✅ Page 2 saved as pdf_images\page_2.png
✅ Page 3 saved as pdf_images\page_3.png
✅ Page 4 saved as pdf_images\page_4.png
✅ Page 5 saved as pdf_images\page_5.png
Step 2: Extracting text from images...
Processing page 1/5...
Processing page 2/5...
Processing page 3/5...
Processing page 4/5...
Processing page 5/5...
🗑️ Deleted temporary folder: pdf_images
Extracted text from Return Policy.pdf added.

✅ Extraction Complete! All text saved to multipdf_merge_combined_extracted_text.txt
✅ Successfully embedded 9 chunks from multipdf_merge_combined_extracted_text.txt
🤖 Chatbot (qwen3:8b) is ready! Type 'exit' to quit.

🤖 Bot: 📄 Chunk 4: Page Break --- 8/25/25, 5.35 PM Calmly Writer Online Refunds